
# Sustainable Resource Management 
 # Topic: Smart Irrigation System

 ## Next-Gen





# Proposed Solution

The AquaSmart irrigation system leverages advanced technologies to optimize water usage in agriculture. The proposed solution includes the following key components:

## 1. IoT Sensor Integration
- **Temperature Sensors**: Monitor ambient temperature to assess the environmental conditions affecting irrigation needs.
- **Soil Moisture Sensors**: Measure the moisture level in the soil to determine when irrigation is necessary.
- **Humidity Sensors**: Track humidity levels, which can influence evaporation rates and plant water requirements.

## 2. AI Model for Rain Prediction
- The system utilizes an AI model that processes Earth observation data to predict rainfall on specific days. This model will help identify the likelihood of precipitation, allowing for more efficient scheduling of irrigation.

## 3. Smart Irrigation Scheduling
- By combining data from the IoT sensors and the AI model, AquaSmart can determine the optimal times for irrigation. This ensures that water is only used when necessary, preventing waste and conserving resources.

## 4. User-Friendly Interface
- The system includes a user-friendly interface that allows farmers to monitor sensor data in real-time, adjust irrigation schedules, and receive notifications about weather conditions.

## Benefits
- **Water Conservation**: Reduces water waste by ensuring irrigation occurs only when necessary.
- **Increased Crop Yield**: Optimized watering schedules can lead to healthier plants and improved agricultural productivity.
- **Cost Efficiency**: Lower operational costs for farmers through efficient water usage.



## List of Components

1. IoT devices
- GAIA basic mcu
- GAIA basic carrier board
- GAIA LoRa Receiver

2. External sensors
- Temperature and humidity sensor: DHT22
- Capacitive soil moisture sensor (NextGen)
- Resistive soil moisture sensor (GeoInnovators)

3. Earth Obsevation (EO) data

- Website
- satellite
- collected parameters


#  Cloud Deployment

## Data reception: MQTT broker

In [ ]:
import paho.mqtt.client as mqtt
import mysql.connector
from mysql.connector import errorcode
import json


# Configuration for connecting to MySQL
config = {
    'user': 'new_user',  # Replace with your MySQL username
    'password': 'your_password',  # Replace with your MySQL password
    'host': 'localhost',  # Replace with your MySQL host, if not localhost
    'database': 'GaiaBase'
}



# Define the callback function for when a message is received
def on_message(client, userdata, message):
    insert_json_data(str(message.payload.decode("utf-8")))
    print(f"Topic: {message.topic}\nMessage: {message.payload.decode()}")

# Define the callback function for when the client connects to the broker
def on_connect(client, userdata, flags, rc):

    # Connect to MySQL server
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # Create database and table if not exists
    create_database_if_not_exists(cursor)
    create_table_if_not_exists(cursor)

    print(f"Connected with result code {rc}")
    client.subscribe("gaia-lora-receiver/outgoing")


    cursor.close()
    conn.close()


def create_database_if_not_exists(cursor):
    try:
        cursor.execute("CREATE DATABASE IF NOT EXISTS GaiaBase")
        print("Database 'GaiaBase' checked/created.")
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")
        exit(1)

def create_table_if_not_exists(cursor):
    cursor.execute("USE GaiaBase")
    table_creation_query = """
    CREATE TABLE IF NOT EXISTS sensor_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        soil_moisture INT NOT NULL,
        temperature FLOAT NOT NULL,
        humidity FLOAT NOT NULL,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    try:
        cursor.execute(table_creation_query)
        print("Table 'sensor_data' checked/created.")
    except mysql.connector.Error as err:
        print(f"Failed creating table: {err}")
        exit(1)

def insert_json_data(json_data):
    try:
        # Connect to MySQL server
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()


        # Parse JSON data
        data = json.loads(json_data)
        soil_moisture = data.get('soil_moisture')
        temperature = data.get('temperature')
        humidity = data.get('humidity')

        # Insert data into table
        insert_query = """
        INSERT INTO sensor_data (soil_moisture, temperature, humidity)
        VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (soil_moisture, temperature, humidity))
        conn.commit()
        print("Data inserted successfully.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        conn.close()



# Create an MQTT client instance
client = mqtt.Client()

# Set the callback functions
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker running on the same server
client.connect("localhost", 1883, 60)

# Start the loop to process network traffic and dispatch callbacks
client.loop_forever()


## IoT Data Base

In [ ]:
# Step 1: Install the MySQL connector
!pip install mysql-connector-python

# Step 2: Import Libraries
import mysql.connector
import pandas as pd

# Function to connect to the MySQL database and fetch data
def fetch_sensor_data():
    try:
        # Step 3: Establish Connection to the MySQL Database
        db_connection = mysql.connector.connect(
            host="localhost",  # Change this if your MySQL server is hosted elsewhere
            user="new_user",
            password="your-password",
            database="GaiaBase"
        )

        # Step 4: Query the Database
        cursor = db_connection.cursor()
        cursor.execute("SELECT * FROM sensor_data")
        data = cursor.fetchall()

        # Step 5: Get Column Names and Convert to DataFrame
        column_names = [i[0] for i in cursor.description]
        sensor_data_df = pd.DataFrame(data, columns=column_names)

        # Close the cursor and connection
        cursor.close()
        db_connection.close()

        # Return the DataFrame for visualization
        return sensor_data_df

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Fetch and visualize the data
sensor_data_df = fetch_sensor_data()

# Check if data was retrieved successfully
if sensor_data_df is not None:
    # Display the DataFrame
    display(sensor_data_df)  # Use display for better formatting in Jupyter Notebook
else:
    print("Failed to retrieve data.")


## Earth Observation (EO) data acquisition

## Earth Observation Parameters for Rainfall Prediction

To predict rainfall, we will utilize the following Earth observation parameters:

1. **PRECTOTCORR**: Total corrected precipitation (mm).
2. **PS**: Surface pressure (Pa).
3. **RH2M**: Relative humidity at 2 meters above ground level (%).
4. **T2MDEW**: Dew point temperature at 2 meters above ground level (°C).
5. **T2M**: Temperature at 2 meters above ground level (°C).
6. **WS10M**: Wind speed at 10 meters above ground level (m/s).
7. **WD10M**: Wind direction at 10 meters above ground level (degrees).
8. **ALLSKY_SFC_SW_DWN**: Downward shortwave radiation at the surface (W/m²).

These parameters will be used in our predictive model to determine the likelihood of rainfall, which will inform our irrigation scheduling and help conserve water resources.


Building an AI Model to Predict Rain from Scratch (Collecting the Data, Training the Model, Testing) - Link Kaggle https://www.kaggle.com/models/jawhersadok01/rain_prediction

We extracted the parameters from the website https://power.larc.nasa.gov.

In [ ]:
def get_meteo_data(latitude, longitude, start_date, end_date):
    # Define the NASA POWER API URL
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=PRECTOTCORR,PS,RH2M,T2MDEW,T2M,WS10M,WD10M,ALLSKY_SFC_SW_DWN&community=RE&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=JSON"
    
    # Send a GET request to fetch the data
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

We implemented the AI model that we built, rain_model.pkl, to predict rainfall for the day after the current day.

In [ ]:
current_date = datetime.utcnow()
start_date = current_date
end_date = current_date

data = get_meteo_data(latitude, longitude, start_date, end_date)

if data:
    try:
        dates = list(data['properties']['parameter']['PRECTOTCORR'].keys())
        
        required_parameters = ['PRECTOTCORR', 'PS', 'RH2M', 'T2MDEW', 'T2M', 'WS10M', 'WD10M', 'ALLSKY_SFC_SW_DWN']
        default_values = {
            'PRECTOTCORR': 0.0,
            'PS': 1013.25,
            'RH2M': 50.0,
            'T2MDEW': 10.0,
            'T2M': 20.0,
            'WS10M': 3.0,
            'WD10M': 180.0,
            'ALLSKY_SFC_SW_DWN': 200.0
        }

        parameters_data = {param: data['properties']['parameter'].get(param, {}) for param in required_parameters}
        
        df = pd.DataFrame({
            date: {
                param: parameters_data[param].get(date, default_values[param]) if parameters_data[param].get(date, -999.00) != -999.00 else default_values[param]
                for param in required_parameters
            }
            for date in dates
        }).T

        df.index.name = 'Date'

    except KeyError as e:
        print(f"Error extracting data: {e}")
else:
    print("No data returned from the API.")

# Get the first row of the DataFrame
first_row = df.iloc[0]

# Extract the date and parameters
date_str = first_row.name
date = pd.to_datetime(date_str, format='%Y%m%d')
year = date.year
month = date.month
day = date.day
day_of_week = date.day_name()  
is_weekend = day_of_week in ['Saturday', 'Sunday']
week_of_year = date.isocalendar()[1]  
quarter = date.quarter

# Extract parameters
PRECTOTCORR = first_row['PRECTOTCORR']
PS = first_row['PS']
RH2M = first_row['RH2M']
T2MDEW = first_row['T2MDEW']
T2M = first_row['T2M']
WS10M = first_row['WS10M']
WD10M = first_row['WD10M']
ALLSKY_SFC_SW_DWN = first_row['ALLSKY_SFC_SW_DWN']

# Encode the day_of_week
label_encoder = LabelEncoder()
day_of_week_encoded = label_encoder.fit_transform([day_of_week])[0]
# Print the extracted values
print(f"Date: {date_str}")
print(f"Year: {year}")
print(f"Month: {month}")
print(f"Day: {day}")
print(f"Day of the Week: {day_of_week}")
print(f"Is Weekend: {is_weekend}")
print(f"Week of the Year: {week_of_year}")
print(f"Quarter: {quarter}")
print(f"PRECTOTCORR: {PRECTOTCORR}")
print(f"PS: {PS}")
print(f"RH2M: {RH2M}")
print(f"T2MDEW: {T2MDEW}")
print(f"T2M: {T2M}")
print(f"WS10M: {WS10M}")
print(f"WD10M: {WD10M}")
print(f"ALLSKY_SFC_SW_DWN: {ALLSKY_SFC_SW_DWN}")
# Prepare new data for prediction
model = joblib.load('rain_model.pkl')
new_data = [[
    PRECTOTCORR,
    PS,
    RH2M,
    T2MDEW,
    T2M,
    WS10M,
    WD10M,
    ALLSKY_SFC_SW_DWN,
    year,
    month,
    day,
    day_of_week_encoded,  # Use the encoded value
    is_weekend,
    week_of_year,
    quarter
]]

new_data_df = pd.DataFrame(new_data, columns=[
    'PRECTOTCORR',
    'PS',
    'RH2M',
    'T2MDEW',
    'T2M',
    'WS10M',
    'WD10M',
    'ALLSKY_SFC_SW_DWN',
    'year',
    'month',
    'day',
    'day_of_week',  # Note: This is now encoded
    'is_weekend',
    'week_of_year',
    'quarter'
])

# Make a prediction
predicted_rain = model.predict(new_data_df)
if (predicted_rain[0]==0):
    print(" predicted_rain:","NO" )
else:
  print("predicted_rain:","YES"  )


## Irrigation scheduling approach

Our solution operates as follows: when IoT information arrives at the MQTT broker, it is sent to be stored in a MySQL database. We then use the AI model to predict whether it will rain the next day. If rain is predicted, the system will refrain from irrigating; if no rain is expected, the system will proceed with irrigation.

Irrigation will commence at 6 AM. The duration of irrigation is determined by several factors, including the surface area of the land, the desired soil depth for irrigation, and the flow rate of the electric valve. After performing the necessary calculations, we will determine the total time required to irrigate the entire area.

Using the IoT data from the database, we will also calculate the average humidity, temperature, and soil moisture. Based on these values, we may adjust the calculated irrigation time, adding extra time as needed to optimize conditions for the crops.









## the complete code 

In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import paho.mqtt.client as mqtt
import mysql.connector
import joblib
from sklearn.preprocessing import LabelEncoder
################################################################################################################################
# Database connection parameters
db_config = {
    'user': 'new_user',
    'password': 'your_password',
    'host': 'localhost',  # e.g., 'localhost' or your cloud MySQL server
    'database': 'GaiaBase'
}

# Function to fetch data from MySQL database
def fetch_data(cursor):
    query = """
    SELECT temperature, soil_moisture, humidity, timestamp
    FROM sensor_data
    WHERE timestamp >= %s AND timestamp < %s
    """
   
    # Set the time range for the data collection
    start_time = datetime.now().replace(hour=6, minute=0, second=0, microsecond=0)
    end_time = start_time + timedelta(days=1)

    cursor.execute(query, (start_time, end_time))
    return cursor.fetchall()

# Function to calculate average values
def calculate_averages(data):
    total_temp = 0
    total_moisture = 0
    total_humidity = 0
    count = len(data)
   
    for row in data:
        total_temp += row[0]  # Temperature
        total_moisture += row[1]  # Soil Moisture
        total_humidity += row[2]  # Humidity

    return (total_temp / count, total_moisture / count, total_humidity / count) if count > 0 else (0, 0, 0)

# Function to determine irrigation time needed based on conditions
def calculate_irrigation_time(avg_temp, avg_moisture, avg_humidity):
    # Calculate the volume of water needed based on the surface area
    area = 20 * 20  # in square meters
    depth = 0.1  # Desired depth of water in meters (10 cm)
   
    volume_needed = area * depth  # in cubic meters

    # Determine the flow rate of the electrovane
    flow_rate = 9  # flow rate in cubic meters per hour
   
    # Calculate time in hours needed for irrigation based on volume needed
    time_needed_hours = volume_needed / flow_rate  # in hours
    time_needed_minutes = time_needed_hours * 60  # converting hours to minutes
   
    # Additional conditions can be included to modify the irrigation duration
    if avg_moisture > 800:  # Increase time needed if moisture is low
        time_needed_minutes += 30  # add 30 minutes if moisture is low
    elif avg_humidity < 50:  # Increase time needed if humidity is low
        time_needed_minutes += 15  # add 15 minutes if humidity is low

    return time_needed_minutes * 60 * 1000  # converting minutes to milliseconds

def main():
    # Connect to the MySQL database
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    try:
        # Fetch the data
        data = fetch_data(cursor)
        avg_temp, avg_moisture, avg_humidity = calculate_averages(data)
       
        print(f"Average Temperature: {avg_temp:.2f} °C")
        print(f"Average Soil Moisture: {avg_moisture:.2f} %")
        print(f"Average Humidity: {avg_humidity:.2f} %")

        # Calculate the irrigation time needed in milliseconds
        irrigation_time = calculate_irrigation_time(avg_temp, avg_moisture, avg_humidity)
       
        # Schedule the irrigation system
        print(f"Time needed for irrigation: {irrigation_time:.0f} milliseconds")
        return(int(irrigation_time))
        # Here you can add code to activate the irrigation system
        # e.g., control the electrovane based on the irrigation_time

    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    irrigation_time = main()
############################################################################################################################

def get_meteo_data(latitude, longitude, start_date, end_date):
    # Define the NASA POWER API URL
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=PRECTOTCORR,PS,RH2M,T2MDEW,T2M,WS10M,WD10M,ALLSKY_SFC_SW_DWN&community=RE&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=JSON"
    
    # Send a GET request to fetch the data
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

# Example usage§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§
latitude = 36.892359973193614
longitude = 10.187902646026297

# Get the current date§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§§
current_date = datetime.utcnow()
start_date = current_date
end_date = current_date

data = get_meteo_data(latitude, longitude, start_date, end_date)

if data:
    try:
        dates = list(data['properties']['parameter']['PRECTOTCORR'].keys())
        
        required_parameters = ['PRECTOTCORR', 'PS', 'RH2M', 'T2MDEW', 'T2M', 'WS10M', 'WD10M', 'ALLSKY_SFC_SW_DWN']
        default_values = {
            'PRECTOTCORR': 0.0,
            'PS': 1013.25,
            'RH2M': 50.0,
            'T2MDEW': 10.0,
            'T2M': 20.0,
            'WS10M': 3.0,
            'WD10M': 180.0,
            'ALLSKY_SFC_SW_DWN': 200.0
        }

        parameters_data = {param: data['properties']['parameter'].get(param, {}) for param in required_parameters}
        
        df = pd.DataFrame({
            date: {
                param: parameters_data[param].get(date, default_values[param]) if parameters_data[param].get(date, -999.00) != -999.00 else default_values[param]
                for param in required_parameters
            }
            for date in dates
        }).T

        df.index.name = 'Date'

    except KeyError as e:
        print(f"Error extracting data: {e}")
else:
    print("No data returned from the API.")

# Get the first row of the DataFrame
first_row = df.iloc[0]

# Extract the date and parameters
date_str = first_row.name
date = pd.to_datetime(date_str, format='%Y%m%d')
year = date.year
month = date.month
day = date.day
day_of_week = date.day_name()  
is_weekend = day_of_week in ['Saturday', 'Sunday']
week_of_year = date.isocalendar()[1]  
quarter = date.quarter

# Extract parameters
PRECTOTCORR = first_row['PRECTOTCORR']
PS = first_row['PS']
RH2M = first_row['RH2M']
T2MDEW = first_row['T2MDEW']
T2M = first_row['T2M']
WS10M = first_row['WS10M']
WD10M = first_row['WD10M']
ALLSKY_SFC_SW_DWN = first_row['ALLSKY_SFC_SW_DWN']

# Encode the day_of_week
label_encoder = LabelEncoder()
day_of_week_encoded = label_encoder.fit_transform([day_of_week])[0]
# Print the extracted values
print(f"Date: {date_str}")
print(f"Year: {year}")
print(f"Month: {month}")
print(f"Day: {day}")
print(f"Day of the Week: {day_of_week}")
print(f"Is Weekend: {is_weekend}")
print(f"Week of the Year: {week_of_year}")
print(f"Quarter: {quarter}")
print(f"PRECTOTCORR: {PRECTOTCORR}")
print(f"PS: {PS}")
print(f"RH2M: {RH2M}")
print(f"T2MDEW: {T2MDEW}")
print(f"T2M: {T2M}")
print(f"WS10M: {WS10M}")
print(f"WD10M: {WD10M}")
print(f"ALLSKY_SFC_SW_DWN: {ALLSKY_SFC_SW_DWN}")
# Prepare new data for prediction
model = joblib.load('rain_model.pkl')
new_data = [[
    PRECTOTCORR,
    PS,
    RH2M,
    T2MDEW,
    T2M,
    WS10M,
    WD10M,
    ALLSKY_SFC_SW_DWN,
    year,
    month,
    day,
    day_of_week_encoded,  # Use the encoded value
    is_weekend,
    week_of_year,
    quarter
]]

new_data_df = pd.DataFrame(new_data, columns=[
    'PRECTOTCORR',
    'PS',
    'RH2M',
    'T2MDEW',
    'T2M',
    'WS10M',
    'WD10M',
    'ALLSKY_SFC_SW_DWN',
    'year',
    'month',
    'day',
    'day_of_week',  # Note: This is now encoded
    'is_weekend',
    'week_of_year',
    'quarter'
])

# Make a prediction
predicted_rain = model.predict(new_data_df)
if (predicted_rain[0]==0):
    print(" predicted_rain:","NO" )
else:
  print("predicted_rain:","YES"  )



####################################################################################################################################################################
if(predicted_rain[0]==0):
  T = str (irrigation_time)
else:
  T = "0"

####################################################################################################################################################################
# Define the callback function for when a message is published
def on_publish(client, userdata, mid):
    print("Message published with mid: {}".format(mid))

# Define the callback for when the client connects to the broker
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully!")
    else:
        print("Connection failed with code", rc)

def sendduration():
    # Create an MQTT client instance
    client = mqtt.Client()

    # Assign the callbacks
    client.on_connect = on_connect
    client.on_publish = on_publish

    # Connect to the MQTT broker (update the host and port as necessary)
    client.connect("localhost", 1883, 60)

    # Publish the message
    topic = "gaia-lora-receiver/incoming"
    result = client.publish(topic, T)

    # Check if the publish was successful
    if result.rc == mqtt.MQTT_ERR_SUCCESS:
        print("Message sent successfully.")
    else:
        print("Failed to send message.")

    # Loop to process network traffic and callbacks
    client.loop_start()

    # Optional: wait for a moment before disconnecting
    import time
    time.sleep(1)

    # Disconnect from the broker
    client.loop_stop()
    client.disconnect()


sendduration()


we set a timer to automate the process so that it repeats every day.

In [ ]:
from timezonefinder import TimezoneFinder
from datetime import datetime
import pytz
import subprocess
import time

def is_time_between_6_and_630(local_time_str):
    # Define the time range in HH:MM format
    start_time = "06:00"
    end_time = "06:30"

    # Convert the time strings into datetime objects
    time_format = '%H:%M'
    local_time = datetime.strptime(local_time_str, time_format)
    start_time_dt = datetime.strptime(start_time, time_format)
    end_time_dt = datetime.strptime(end_time, time_format)

    # Check if the local_time is between start_time and end_time
    return start_time_dt <= local_time <= end_time_dt
def get_local_time(latitude, longitude):
    # Initialize TimezoneFinder object
    tf = TimezoneFinder()

    # Find the timezone of the given coordinates
    timezone_str = tf.timezone_at(lat=latitude, lng=longitude)

    if timezone_str:
        # Get the timezone object
        timezone = pytz.timezone(timezone_str)

        # Get the current time in that timezone
        local_time = datetime.now(timezone)

        # Return the time in HH:MM format
        return local_time.strftime('%H:%M')
    else:
        return "Timezone not found"

# Example usage
latitude = 36.892359973193614
longitude = 10.187902646026297
 

while True :
  # usage:
  local_time_str = get_local_time(latitude, longitude)
  result = is_time_between_6_and_630(local_time_str)
  if result :
    subprocess.run(["python", "solution.py"])
    time.sleep(1800)  # Pause for 30 minutes before the next iteration 